In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from numpy.random import RandomState
from sklearn import metrics
from sklearn.utils import shuffle
import os

In [57]:
#labeled_data = {'source':[],'type':[],'text':[],'stance':[]}
#unlabeled_data = {'source':[],'type':[],'text':[]}

# Dallas labeled news

In [10]:
import json
lines = []
with open('all.pro-anti.sentences.jsonlist') as f:
    for line in f:
        lines.append(json.loads(line))

In [11]:
len(lines)

37105

In [83]:
#lines[10]

In [38]:
Counter([l['primary_tone'].lower() for l in lines])

Counter({'pro': 30917, 'anti': 6188})

In [39]:
tone2label = {'pro':'agree','anti':'disagree'}

In [58]:
labeled_data['source'] = [l['article_id'] + ' [SEP] ' + str(l['sent_index']) for l in lines]
labeled_data['type'] = ['dallas_news']*len(lines)
labeled_data['text'] = [l['text'] for l in lines]
labeled_data['stance'] = [tone2label[l['primary_tone'].lower()] for l in lines]
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,climate_change1.0-12174 [SEP] 0,dallas_news,It's about seeing the forest and the trees in ...,agree
1,climate_change1.0-12174 [SEP] 1,dallas_news,Two of his colleagues dangled on ropes 100 fee...,agree
2,climate_change1.0-12174 [SEP] 2,dallas_news,Steve Sillett could hardly contain his excitem...,agree
3,climate_change1.0-12174 [SEP] 3,dallas_news,"""This site is just juicy.",agree
4,climate_change1.0-12174 [SEP] 4,dallas_news,"It's loaded,"" said Sillett, gazing up at the a...",agree
5,climate_change1.0-12174 [SEP] 5,dallas_news,"""There are amazing lichens up there.",agree
6,climate_change1.0-12174 [SEP] 6,dallas_news,There are aphids being hunted by ladybugs.,agree
7,climate_change1.0-12174 [SEP] 7,dallas_news,I found a sizable dogwood tree growing from a ...,agree
8,climate_change1.0-12174 [SEP] 8,dallas_news,"The foggy, lost worlds atop the world's bigges...",agree
9,climate_change1.0-12174 [SEP] 9,dallas_news,Concerned that rising temperatures across Cali...,agree


# Dallas unlabeled news

In [17]:
all_sents = []
with open('all.sentences.jsonlist') as f:
    for line in f:
        all_sents.append(json.loads(line))

In [18]:
len(all_sents)

126100

In [20]:
'climate_change1.0-6365 [SEP] 0' in labeled_data_df['source']

False

In [84]:
#all_sents[1]

In [178]:
unlabeled_data['source'] = [l['article_id'] + ' [SEP] ' + str(l['sent_index']) 
                               for l in all_sents]
unlabeled_data['type'] = ['dallas_news']*len(all_sents)
unlabeled_data['text'] = [l['text'] for l in all_sents]
unlabeled_data_df = pd.DataFrame(unlabeled_data)
unlabeled_data_df.head(10)

,source,type,text
0,climate_change1.0-6365 [SEP] 0,dallas_news,"No Accord, and Little Optimism, at Bonn Talks ..."
1,climate_change1.0-6365 [SEP] 1,dallas_news,Delegates from 143 countries ended their preli...
2,climate_change1.0-6365 [SEP] 2,dallas_news,The United States' long-awaited proposal for c...
3,climate_change1.0-6365 [SEP] 3,dallas_news,And there is no sign that American negotiators...
4,climate_change1.0-6365 [SEP] 4,dallas_news,"Still, the Americans themselves said there was..."
5,climate_change1.0-6365 [SEP] 5,dallas_news,"Meanwhile, several small-scale but higher-leve..."
6,climate_change1.0-6365 [SEP] 6,dallas_news,The goal of the conference is to revise the cl...
7,climate_change1.0-6365 [SEP] 7,dallas_news,"""There are enough ideas out there to have an a..."
8,climate_change1.0-6365 [SEP] 8,dallas_news,"""We just don't see all the kinds of coalescenc..."
9,climate_change1.0-6365 [SEP] 9,dallas_news,"On the surface, the American proposal looks li..."


# SemEval tweets

In [30]:
targets = []
with open(os.getcwd()+'/semeval2016-task6-testdata-gold/SemEval2016-Task6-subtaskB-testdata\
-gold.txt') as f:
    semeval_b = f.readlines()
    for line in semeval_b[1:]:
        id_,target,tweet,stance = line.strip().split('\t')
        targets.append(target)

In [32]:
from collections import Counter
Counter(targets)

Counter({'Donald Trump': 707})

In [28]:
semeval_tweets = []
with open(os.getcwd()+'/semeval2016-task6-testdata-gold/SemEval2016-Task6-subtaskA-testdata\
-gold.txt') as f:
    semeval_a = f.readlines()
    for line in semeval_a[1:]:
        id_,target,tweet,stance = line.strip().split('\t')
        if target == 'Climate Change is a Real Concern':
            semeval_tweets.append(line.strip().split('\t'))

In [29]:
len(semeval_tweets)

169

In [33]:
from collections import Counter
Counter([x[-1] for x in semeval_tweets])

Counter({'NONE': 35, 'FAVOR': 123, 'AGAINST': 11})

In [34]:
tweetstance2label = {'NONE':'neutral','FAVOR':'agree','AGAINST':'disagree'}

In [59]:
labeled_data['source'].extend([l[0] for l in semeval_tweets])
labeled_data['type'].extend(['semeval_tweets']*len(semeval_tweets))
labeled_data['text'].extend([l[2] for l in semeval_tweets])
labeled_data['stance'].extend([tweetstance2label[l[3]] for l in semeval_tweets])
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,climate_change1.0-12174 [SEP] 0,dallas_news,It's about seeing the forest and the trees in ...,agree
1,climate_change1.0-12174 [SEP] 1,dallas_news,Two of his colleagues dangled on ropes 100 fee...,agree
2,climate_change1.0-12174 [SEP] 2,dallas_news,Steve Sillett could hardly contain his excitem...,agree
3,climate_change1.0-12174 [SEP] 3,dallas_news,"""This site is just juicy.",agree
4,climate_change1.0-12174 [SEP] 4,dallas_news,"It's loaded,"" said Sillett, gazing up at the a...",agree
5,climate_change1.0-12174 [SEP] 5,dallas_news,"""There are amazing lichens up there.",agree
6,climate_change1.0-12174 [SEP] 6,dallas_news,There are aphids being hunted by ladybugs.,agree
7,climate_change1.0-12174 [SEP] 7,dallas_news,I found a sizable dogwood tree growing from a ...,agree
8,climate_change1.0-12174 [SEP] 8,dallas_news,"The foggy, lost worlds atop the world's bigges...",agree
9,climate_change1.0-12174 [SEP] 9,dallas_news,Concerned that rising temperatures across Cali...,agree


In [43]:
labeled_data_df.tail(10)

,source,type,text,stance
37264,10380,semeval_tweets,@4EvaMartino Everyone should learn this recip...,agree
37265,10381,semeval_tweets,Rec high temp for this date in Charlotte is 10...,agree
37266,10382,semeval_tweets,"Lot's of time, class over! @nationaljournal #S...",neutral
37267,10383,semeval_tweets,@TheLibertyEagle Oh for the love of Pete! I th...,neutral
37268,10384,semeval_tweets,If complaining about irrelevant problems were ...,neutral
37269,10385,semeval_tweets,"@LiberalAus - ""All"" you folks #owe me an #offi...",neutral
37270,10386,semeval_tweets,"Maybe we should start ""labelling every #produc...",agree
37271,10387,semeval_tweets,If we touch #Antarctica for one more #reason o...,agree
37272,10388,semeval_tweets,Wind power helped stabilize Texas' power grid ...,agree
37273,10389,semeval_tweets,"It's so cold and windy here in Sydney, I just ...",agree


In [46]:
stance_test = pd.read_csv(open(os.getcwd()+'/StanceDataset/test.csv','rU'),
                          encoding='utf-8', engine='c')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.


In [47]:
stance_test = stance_test[stance_test['Target'] == 'Climate Change is a Real Concern']

In [48]:
stance_test.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
220,Closed door session begins. More after they de...,Climate Change is a Real Concern,NONE,3. The tweet is not explicitly expressing opi...,other
221,What is the #energiewende agenda? Why are they...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
222,@climasphere Stocker: Fish catch potential cou...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
223,"...a longer memory, and a sterner sense of jus...",Climate Change is a Real Concern,NONE,2. The tweet does NOT expresses opinion about ...,other
224,"Boni ""Future Sea level rise will not be unifor...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,other


In [60]:
labeled_data['source'].extend(list(stance_test.index))
labeled_data['type'].extend(['semeval_test_tweets']*len(stance_test))
labeled_data['text'].extend([stance_test.loc[l]['Tweet'] for l in stance_test.index])
labeled_data['stance'].extend([tweetstance2label[stance_test.loc[l]['Stance']] 
                               for l in stance_test.index])
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,climate_change1.0-12174 [SEP] 0,dallas_news,It's about seeing the forest and the trees in ...,agree
1,climate_change1.0-12174 [SEP] 1,dallas_news,Two of his colleagues dangled on ropes 100 fee...,agree
2,climate_change1.0-12174 [SEP] 2,dallas_news,Steve Sillett could hardly contain his excitem...,agree
3,climate_change1.0-12174 [SEP] 3,dallas_news,"""This site is just juicy.",agree
4,climate_change1.0-12174 [SEP] 4,dallas_news,"It's loaded,"" said Sillett, gazing up at the a...",agree
5,climate_change1.0-12174 [SEP] 5,dallas_news,"""There are amazing lichens up there.",agree
6,climate_change1.0-12174 [SEP] 6,dallas_news,There are aphids being hunted by ladybugs.,agree
7,climate_change1.0-12174 [SEP] 7,dallas_news,I found a sizable dogwood tree growing from a ...,agree
8,climate_change1.0-12174 [SEP] 8,dallas_news,"The foggy, lost worlds atop the world's bigges...",agree
9,climate_change1.0-12174 [SEP] 9,dallas_news,Concerned that rising temperatures across Cali...,agree


In [61]:
labeled_data_df.tail(10)

,source,type,text,stance
37433,379,semeval_test_tweets,@4EvaMartino Everyone should learn this recip...,agree
37434,380,semeval_test_tweets,Rec high temp for this date in Charlotte is 10...,agree
37435,381,semeval_test_tweets,"Lot's of time, class over! @nationaljournal #S...",neutral
37436,382,semeval_test_tweets,@TheLibertyEagle Oh for the love of Pete! I th...,neutral
37437,383,semeval_test_tweets,If complaining about irrelevant problems were ...,neutral
37438,384,semeval_test_tweets,"@LiberalAus - ""All"" you folks #owe me an #offi...",neutral
37439,385,semeval_test_tweets,"Maybe we should start ""labelling every #produc...",agree
37440,386,semeval_test_tweets,If we touch #Antarctica for one more #reason o...,agree
37441,387,semeval_test_tweets,Wind power helped stabilize Texas' power grid ...,agree
37442,388,semeval_test_tweets,"It's so cold and windy here in Sydney, I just ...",agree


In [62]:
stance_train = pd.read_csv(open(os.getcwd()+'/StanceDataset/train.csv','rU'),
                          encoding='utf-8', engine='c')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.


In [63]:
stance_train = stance_train[stance_train['Target'] == 'Climate Change is a Real Concern']

In [64]:
stance_train.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
613,"We cant deny it, its really happening. #SemST",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,other
614,RT @cderworiz: Timelines are short. Strategy m...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
615,SO EXCITING! Meaningful climate change action ...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
616,"Delivering good jobs for Albertans, maintainin...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
617,@davidswann says he wants carbon fund to be sp...,Climate Change is a Real Concern,FAVOR,3. The tweet is not explicitly expressing opi...,other


In [65]:
labeled_data['source'].extend(list(stance_train.index))
labeled_data['type'].extend(['semeval_train_tweets']*len(stance_train))
labeled_data['text'].extend([stance_train.loc[l]['Tweet'] for l in stance_train.index])
labeled_data['stance'].extend([tweetstance2label[stance_train.loc[l]['Stance']] 
                               for l in stance_train.index])
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,climate_change1.0-12174 [SEP] 0,dallas_news,It's about seeing the forest and the trees in ...,agree
1,climate_change1.0-12174 [SEP] 1,dallas_news,Two of his colleagues dangled on ropes 100 fee...,agree
2,climate_change1.0-12174 [SEP] 2,dallas_news,Steve Sillett could hardly contain his excitem...,agree
3,climate_change1.0-12174 [SEP] 3,dallas_news,"""This site is just juicy.",agree
4,climate_change1.0-12174 [SEP] 4,dallas_news,"It's loaded,"" said Sillett, gazing up at the a...",agree
5,climate_change1.0-12174 [SEP] 5,dallas_news,"""There are amazing lichens up there.",agree
6,climate_change1.0-12174 [SEP] 6,dallas_news,There are aphids being hunted by ladybugs.,agree
7,climate_change1.0-12174 [SEP] 7,dallas_news,I found a sizable dogwood tree growing from a ...,agree
8,climate_change1.0-12174 [SEP] 8,dallas_news,"The foggy, lost worlds atop the world's bigges...",agree
9,climate_change1.0-12174 [SEP] 9,dallas_news,Concerned that rising temperatures across Cali...,agree


In [66]:
labeled_data_df.tail(10)

,source,type,text,stance
37828,998,semeval_train_tweets,11% reduction is this a joke? Our govt seems t...,neutral
37829,999,semeval_train_tweets,until newyork london frankfurt knee deep in se...,agree
37830,1000,semeval_train_tweets,".@ClimatParis2015, the climate science forum b...",agree
37831,1001,semeval_train_tweets,We need integrated #science with #indigenous k...,agree
37832,1002,semeval_train_tweets,".@whelan60 ""While this debate goes on, yet mor...",neutral
37833,1003,semeval_train_tweets,If sea levels get any higher flooding in lowla...,agree
37834,1004,semeval_train_tweets,If we don't do anything we will be the sad gen...,agree
37835,1005,semeval_train_tweets,Vallaud-Belkacem: Are we the generation who kn...,agree
37836,1006,semeval_train_tweets,"If we do not act, we will be the generation wh...",agree
37837,1007,semeval_train_tweets,The carbon clock is ticking @CarbonBubble #Sur...,agree


# Dallas and Yiwei manually annotated news--not yet added

In [69]:
dallas_labels = []
with open('subsample_annotated_dbc.jsonlist') as f:
    for line in f:
        dallas_labels.append(json.loads(line))

In [70]:
len(dallas_labels)

300

In [71]:
dallas_labels[0]

{'comp_clause': 'that the global warming debate is just like the tobacco wars of the past four decades',
 'full': 'Reading "an early draft" of a federal government report has helped David Ignatius see that the global warming debate is just like the tobacco wars of the past four decades ["Naysay Now, Pay Later," op-ed, Aug. 18].',
 'key': 19565,
 'label': 2}

In [74]:
yiwei_labels = pd.read_csv('subsample_annotated_yiwei.csv',header=None)

In [75]:
yiwei_labels.head()

,0,1,2,3
0,19565,"Reading ""an early draft"" of a federal governme...",that the global warming debate is just like th...,1
1,92670,"We haven’t seen that in the observations,” Myl...","that in the observations , ” Myles Allen , pro...",2
2,152040,"Describing it as a rare hybrid “superstorm,” f...",Sandy was created by an Arctic jet stream wrap...,2
3,191315,JEFFERSON CITY o Ameren Missouri says it must ...,it must decide quickly on how to comply with p...,2
4,250921,There was this exposé in the Times where you f...,what was happening under the Bush administrati...,3


In [75]:
anti_test_data = []
pro_test_data = []
with open(os.getcwd()+'/test_data/anti-cc/test_data.txt') as f:
    for line in f:
        anti_test_data.append(line.split('\t'))
with open(os.getcwd()+'/test_data/pro-cc/test_data.txt') as f:
    for line in f:
        pro_test_data.append(line.split('\t'))

In [76]:
len(anti_test_data)

92804

In [77]:
anti_test_data[0]

['We ’re also doing some avian and marine mammal monitoring',
 '',
 'sent_no_29 of https://www.csmonitor.com/Environment/2010/1012/Will-Google-wind-power-project-harm-wildlife-Depends-on-location.',
 '',
 '“We’re also doing some avian and marine mammal monitoring,” says Rhonda Jackson, a spokesman for Fishermen’s Energy.\n']

In [78]:
len(pro_test_data)

581383

In [81]:
Counter([len(d) for d in pro_test_data])

Counter({5: 286327,
         1: 290241,
         17: 2316,
         6: 113,
         7: 59,
         9: 34,
         29: 2,
         14: 27,
         11: 33,
         8: 12,
         15: 1,
         53: 353,
         273: 56,
         292: 1,
         111: 91,
         106: 464,
         138: 459,
         271: 165,
         103: 209,
         145: 1,
         107: 12,
         60: 5,
         75: 11,
         54: 7,
         68: 6,
         132: 2,
         94: 2,
         128: 5,
         348: 17,
         95: 1,
         57: 1,
         283: 1,
         71: 11,
         354: 13,
         360: 1,
         380: 2,
         401: 1,
         74: 1,
         129: 2,
         69: 17,
         369: 2,
         414: 1,
         70: 4,
         77: 2,
         318: 4,
         117: 14,
         91: 7,
         139: 5,
         121: 8,
         130: 3,
         358: 5,
         362: 2,
         287: 1,
         169: 7,
         186: 4,
         203: 4,
         277: 6,
         382: 5,
      

# Yiwei unlabeled news

In [98]:
all_url_df = pd.read_pickle('/Users/yiweiluo/Dropbox/research/\
QP2/code/Fox_and_friends/all_urls_meta_and_fulltext_df_2020.pkl')

In [99]:
cc_all_url_df = all_url_df[all_url_df['topic'] == 'cc'][all_url_df['bad NYT'] == False]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [100]:
cc_all_url_df.head()

,bad NYT,date,domain,fulltext,stance,title,topic,url
1600,False,None,:///,If you can't sleep as much as you need durin...,anti,None,cc,foxnews.com/lifestyle/climate-change-might-be-...
1601,False,None,:///,Paper says draft document by scientists direct...,anti,None,cc,foxnews.com/opinion/climate-policy-get-ready-f...
1602,False,None,:///,The public's concern about global warming has ...,anti,None,cc,foxnews.com/opinion/physicist-dont-fall-for-th...
1603,False,None,:///,The federal report by dozens of U.S. governmen...,anti,None,cc,foxnews.com/politics/junk-science-studies-behi...
1604,False,None,:///,File photo - A statue of the Virgin Mary is ...,anti,None,cc,foxnews.com/science/10-years-after-hurricane-k...


In [102]:
cc_all_url_df.shape

(17601, 8)

In [181]:
from nltk.tokenize import sent_tokenize
for ix,row in cc_all_url_df.iterrows():
    source = row['url']
    type_ = 'yiwei_news'
    if row['fulltext'] is not None:
        text = sent_tokenize(row['fulltext'])
        unlabeled_data['text'].extend(text)
        unlabeled_data['type'].extend([type_]*len(text))
        unlabeled_data['source'].extend([source]*len(text))

In [192]:
unlabeled_data_df = pd.DataFrame(unlabeled_data)
unlabeled_data_df.head(10)

,source,type,text
0,climate_change1.0-6365 [SEP] 0,dallas_news,"No Accord, and Little Optimism, at Bonn Talks ..."
1,climate_change1.0-6365 [SEP] 1,dallas_news,Delegates from 143 countries ended their preli...
2,climate_change1.0-6365 [SEP] 2,dallas_news,The United States' long-awaited proposal for c...
3,climate_change1.0-6365 [SEP] 3,dallas_news,And there is no sign that American negotiators...
4,climate_change1.0-6365 [SEP] 4,dallas_news,"Still, the Americans themselves said there was..."
5,climate_change1.0-6365 [SEP] 5,dallas_news,"Meanwhile, several small-scale but higher-leve..."
6,climate_change1.0-6365 [SEP] 6,dallas_news,The goal of the conference is to revise the cl...
7,climate_change1.0-6365 [SEP] 7,dallas_news,"""There are enough ideas out there to have an a..."
8,climate_change1.0-6365 [SEP] 8,dallas_news,"""We just don't see all the kinds of coalescenc..."
9,climate_change1.0-6365 [SEP] 9,dallas_news,"On the surface, the American proposal looks li..."


In [193]:
unlabeled_data_df.tail(10)

,source,type,text
1158101,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"and ""do you have a signed ration card from you..."
1158102,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,You can stand by but there's no telling when a...
1158103,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Comrade.
1158104,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Now, a really good high-speed rail system woul..."
1158105,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Let's say something that can travel at 500 mil...
1158106,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Something that can go to every major city, and..."
1158107,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Something that doesn't need new rights of way,..."
1158108,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Something where the rights of way are as free ...
1158109,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,But hoping for something like that is just cra...
1158110,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,https://pjmedia.com/trending/aoc-and-the-magic...


# Clean, dedup, filter labeled and unlabeled data

In [110]:
# Functions for importing & cleaning labeled items
def lower(s):
    return s.lower()

def imports(filename):
    imp = pd.read_pickle(filename)
    imp = imp.drop_duplicates()
    imp['sent_clean'] = imp['sentence'].str.replace('http\S+|www.\S+|pic.twitter.com\S+', '', case=False)
    #imp['sent_clean'] = imp['sentence'].replace('[^A-Za-z0-9 ]+','',regex=True)
    imp['sent_clean'] = imp['sent_clean'].apply(lower)
    return imp

def clean(x):
    return x.replace('http\S+|www.\S+|pic.twitter.com\S+', '')

In [194]:
unlabeled_data_df = unlabeled_data_df.dropna()
unlabeled_data_df['text'] = unlabeled_data_df['text'].apply(clean)
unlabeled_data_df.drop_duplicates(subset= 'text',inplace=True)
unlabeled_data_df = unlabeled_data_df.reset_index()

In [195]:
labeled_data_df = labeled_data_df.dropna()
labeled_data_df['text'] = labeled_data_df['text'].apply(clean)
labeled_data_df.drop_duplicates(subset= 'text',inplace=True)
labeled_data_df = labeled_data_df.reset_index()

In [196]:
labeled_data_df = labeled_data_df.drop(['index'],axis=1)

In [197]:
unlabeled_data_df.shape

(866654, 4)

In [198]:
labeled_data_df.shape

(39228, 4)

In [ ]:
# summary of what's included

In [199]:
unlabeled_data_df.tail(10)

,index,source,type,text
866644,1158101,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"and ""do you have a signed ration card from you..."
866645,1158102,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,You can stand by but there's no telling when a...
866646,1158103,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Comrade.
866647,1158104,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Now, a really good high-speed rail system woul..."
866648,1158105,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Let's say something that can travel at 500 mil...
866649,1158106,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Something that can go to every major city, and..."
866650,1158107,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Something that doesn't need new rights of way,..."
866651,1158108,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Something where the rights of way are as free ...
866652,1158109,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,But hoping for something like that is just cra...
866653,1158110,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,https://pjmedia.com/trending/aoc-and-the-magic...


In [200]:
unlabeled_data_df['type'].value_counts() 

yiwei_news     743344
dallas_news    123310
Name: type, dtype: int64

In [201]:
labeled_data_df

,source,type,text,stance
0,climate_change1.0-12174 [SEP] 0,dallas_news,It's about seeing the forest and the trees in ...,agree
1,climate_change1.0-12174 [SEP] 1,dallas_news,Two of his colleagues dangled on ropes 100 fee...,agree
2,climate_change1.0-12174 [SEP] 2,dallas_news,Steve Sillett could hardly contain his excitem...,agree
3,climate_change1.0-12174 [SEP] 3,dallas_news,"""This site is just juicy.",agree
4,climate_change1.0-12174 [SEP] 4,dallas_news,"It's loaded,"" said Sillett, gazing up at the a...",agree
...,...,...,...,...
39223,round5_batch9_sentidt45,mturk,Although the data series started in 1958 has a...,neutral
39224,round5_batch9_sentidt46,mturk,The Keystone pipeline would unduly worsen clim...,agree
39225,round5_batch9_sentidt47,mturk,Many young adults have become anxious about wh...,neutral
39226,round5_batch9_sentidt48,mturk,"Regardless of the reasons for climate change, ...",agree


In [173]:
labeled_data_df.to_csv('labeled_data_df')

In [202]:
labeled_data_df['stance'].value_counts()

agree       31629
disagree     6534
neutral      1065
Name: stance, dtype: int64

In [203]:
labeled_data_df['type'].value_counts()

dallas_news             36624
mturk                    2040
semeval_train_tweets      395
semeval_tweets            169
Name: type, dtype: int64

In [158]:
dallas_sentence_scores = pd.read_csv('/Users/yiweiluo/scientific-debates/\
cc_stance/MTurk_results/sent_scores_df.tsv',delimiter='\t')
dallas_sentence_scores['max_prob_label'] = dallas_sentence_scores[['disagree',
                                                                  'neutral',
                                                                  'agree']].idxmax(axis=1)

In [161]:
labeled_data['source'].extend(['round{}_batch{}_sentid{}'.format(row['round'],
                                                              row['batch'],
                                                              row['sent_id'])
                              for ix,row in dallas_sentence_scores.iterrows()])
labeled_data['type'].extend(['mturk']*len(dallas_sentence_scores))
labeled_data['text'].extend([row['sentence'] for ix,row in 
                             dallas_sentence_scores.iterrows()])
labeled_data['stance'].extend([row['max_prob_label'] for ix,row in 
                             dallas_sentence_scores.iterrows()])
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,climate_change1.0-12174 [SEP] 0,dallas_news,It's about seeing the forest and the trees in ...,agree
1,climate_change1.0-12174 [SEP] 1,dallas_news,Two of his colleagues dangled on ropes 100 fee...,agree
2,climate_change1.0-12174 [SEP] 2,dallas_news,Steve Sillett could hardly contain his excitem...,agree
3,climate_change1.0-12174 [SEP] 3,dallas_news,"""This site is just juicy.",agree
4,climate_change1.0-12174 [SEP] 4,dallas_news,"It's loaded,"" said Sillett, gazing up at the a...",agree
5,climate_change1.0-12174 [SEP] 5,dallas_news,"""There are amazing lichens up there.",agree
6,climate_change1.0-12174 [SEP] 6,dallas_news,There are aphids being hunted by ladybugs.,agree
7,climate_change1.0-12174 [SEP] 7,dallas_news,I found a sizable dogwood tree growing from a ...,agree
8,climate_change1.0-12174 [SEP] 8,dallas_news,"The foggy, lost worlds atop the world's bigges...",agree
9,climate_change1.0-12174 [SEP] 9,dallas_news,Concerned that rising temperatures across Cali...,agree


In [159]:
#dallas_sentence_scores.to_pickle('dallas_sent_scores.pkl')
dallas_sentence_scores.head()

,Unnamed: 0,round,batch,sent_id,disagree,neutral,agree,sentence,max_prob_label
0,0,1,0,t0,0.004241,0.260963,0.734797,Warmer-than-normal sea surface temperatures ar...,agree
1,1,1,0,t1,0.001548,0.996214,0.002239,We will continue to rely in part on fossil fue...,neutral
2,2,1,0,t10,0.001440,0.996503,0.002057,The actual rise in sea levels measured only 1....,neutral
3,3,1,0,t11,0.996815,0.001588,0.001596,Claims of global warming have been greatly exa...,disagree
4,4,1,0,t12,0.035201,0.959757,0.005042,The Intergovernmental Panel on Climate Change ...,neutral


# Create train / val / test split

In [10]:
all_train = sentence_df[['round','batch','sent_id','disagree','neutral','agree',
                        'max_prob_label','sent_clean']]

print(len(all_train), len(all_train[all_train['max_prob_label']=='agree']), 
      len(all_train[all_train['max_prob_label']=='neutral']),
     len(all_train[all_train['max_prob_label']=='disagree']))

2050 783 863 404


In [11]:
sub_train = all_train[all_train['sent_clean'].str.\
                      contains("climate|change|global|warming")==True]
print(len(sub_train))

1381


In [13]:
300+400+400+500+500

2100

In [14]:
#randomly select tweets that go to validation set or training set
all_train = shuffle(all_train,random_state=123)
all_train = all_train.drop_duplicates('sent_clean')
num_sents = len(all_train)

print(num_sents)
infl_val_agr = all_train[all_train['max_prob_label']=='agree']
infl_val_neut = all_train[all_train['max_prob_label']=='neutral']
infl_val_dis = all_train[all_train['max_prob_label']=='disagree']
print(len(infl_val_agr), len(infl_val_neut), len(infl_val_dis))

train_pct = .9

2042
779 862 401


In [15]:
training_data = all_train[:int(num_sents*train_pct)]

val_data = all_train[int(num_sents*train_pct):]

print("Num total sentences", num_sents,
      "\n Num train sentences", len(training_data), 
      "\n Num validation sentences", len(val_data))

Num total sentences 2042 
 Num train sentences 1837 
 Num validation sentences 205


In [16]:
training_data.to_csv(os.getcwd()+'/data_for_classifier/dedup_training_data.csv', sep=',')
val_data.to_csv(os.getcwd()+'/data_for_classifier/dedup_val_data.csv', sep=',')

In [2]:
import os
os.getcwd()

'/Users/yiweiluo/scientific-debates/cc_stance/MTurk_results'